In [2]:
import pandas as pd
import numpy as np
import os, zipfile
import subprocess

In [24]:
# compare standalone models for binary classification
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
import lightgbm as lgbm
from sklearn.ensemble import StackingClassifier
from matplotlib import pyplot

In [18]:
import category_encoders as ce

In [4]:
dir_name = 'D:\\1mfti\ml\Home-Credit-Default-Risk'
extension = ".zip"

os.chdir(dir_name)

In [5]:
train = pd.read_csv('data/application_train.csv')
test = pd.read_csv('data/application_test.csv')

In [42]:
train['TARGET'].value_counts()

0    282686
1     24825
Name: TARGET, dtype: int64

In [8]:
cat_cols = list(train.select_dtypes(include=['object']).columns)

In [9]:
cat_cols

['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'ORGANIZATION_TYPE',
 'FONDKAPREMONT_MODE',
 'HOUSETYPE_MODE',
 'WALLSMATERIAL_MODE',
 'EMERGENCYSTATE_MODE']

In [10]:
num_cols = list(train.select_dtypes(include=['int64', 'float64']).columns)

In [20]:
train[num_cols]

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,-3648.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,-1186.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,-4260.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,-9833.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,-4311.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,157500.0,254700.0,27558.0,225000.0,0.032561,-9327,-236,-8456.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,72000.0,269550.0,12001.5,225000.0,0.025164,-20775,365243,-4388.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,153000.0,677664.0,29979.0,585000.0,0.005002,-14966,-7921,-6737.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307509,456254,0,171000.0,370107.0,20205.0,319500.0,0.005313,-11961,-4786,-2562.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
num_cols

['SK_ID_CURR',
 'TARGET',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'OWN_CAR_AGE',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'CNT_FAM_MEMBERS',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'HOUR_APPR_PROCESS_START',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'YEARS_BEGINEXPLUATATION_AVG',
 'YEARS_BUILD_AVG',
 'COMMONAREA_AVG',
 'ELEVATORS_AVG',
 'ENTRANCES_AVG',
 'FLOORSMAX_AVG',
 'FLOORSMIN_AVG',
 'LANDAREA_AVG',
 'LIVINGAPARTMENTS_AVG',
 'LIVINGAREA_AVG',
 'NONLIVINGAPARTMENTS_AVG',
 'NONLIVINGAREA_AVG',
 'APARTMENTS_MODE',
 'BASEMENTAR

In [15]:
num_cols.remove('TARGET')

In [76]:
def preprocess(train1, test1):

    train1 = train1.fillna(-10)
    test1 = test1.fillna(-10)
    train1[num_cols] = train1[num_cols].apply(lambda x: pd.cut(x, bins=4, labels=False))
    test1[num_cols] = test1[num_cols].apply(lambda x: pd.cut(x, bins=4, labels=False))
    WOE_encoder = ce.WOEEncoder()
    train1[cat_cols] = WOE_encoder.fit_transform(train1[cat_cols], train1['TARGET'])
    test1[cat_cols] = WOE_encoder.transform(train1[cat_cols])
    
#     train[num_cols] = pd.cut(train[num_cols], bins=4, labels=False)
#     test[num_cols] = pd.cut(test[num_cols], bins=4, labels=False)
    return train1, test1

In [30]:
def get_kaggle_results():
    p = subprocess.Popen("kaggle competitions submissions  -c home-credit-default-risk", stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
    print(output.decode())

In [91]:
train_norm, test_norm = preprocess(train, test)

C:\Python\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [78]:
X_train = train_norm.drop('TARGET', axis=1)
y_train = train_norm['TARGET']
X_test = test_norm

In [86]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN

In [117]:
ada = RandomUnderSampler(random_state=43)
X_res, y_res = ada.fit_resample(X_train, y_train)

In [92]:
sample = ['CNT_CHILDREN',
'AMT_INCOME_TOTAL',
'DAYS_EMPLOYED',
'OWN_CAR_AGE',
'TOTALAREA_MODE',
'DAYS_LAST_PHONE_CHANGE', 'CODE_GENDER',
'FLAG_OWN_CAR', 'NAME_INCOME_TYPE', 
                      'NAME_EDUCATION_TYPE', 
                      'NAME_FAMILY_STATUS', 
                      'NAME_HOUSING_TYPE', 
                      'REGION_RATING_CLIENT', 
                      'ORGANIZATION_TYPE', ]

In [126]:
LR_model_2 = LogisticRegression(C=0.0001, class_weight = 'balanced',max_iter=200, solver = 'saga',penalty = 'elasticnet', l1_ratio = 0.01)

LR_model_2.fit(X_res, y_res)
predict_2 = LR_model_2.predict(X_test)

In [123]:
write_submission_2 = pd.read_csv('data/sample_submission.csv')
write_submission_2['TARGET'] = predict_2
write_submission_2.to_csv('predict_under_sample.csv', index= False)

In [124]:
!kaggle competitions submit -c home-credit-default-risk -f predict_under_sample.csv -m "Message"

Successfully submitted to Home Credit Default Risk



  0%|          | 0.00/476k [00:00<?, ?B/s]
  2%|1         | 8.00k/476k [00:00<00:13, 34.5kB/s]
 20%|##        | 96.0k/476k [00:00<00:01, 293kB/s] 
 42%|####2     | 200k/476k [00:00<00:00, 527kB/s] 
 84%|########4 | 400k/476k [00:00<00:00, 976kB/s]
100%|##########| 476k/476k [00:02<00:00, 180kB/s]


In [125]:
get_kaggle_results()

fileName                          date                 description  status    publicScore  privateScore  
--------------------------------  -------------------  -----------  --------  -----------  ------------  
predict_under_sample.csv          2021-12-02 18:44:24  Message      complete  0.58886      0.59255       
predict_under_sample.csv          2021-12-02 18:43:59  Message      complete  0.60274      0.58831       
predict_under_sample.csv          2021-12-02 18:43:05  Message      complete  0.59447      0.58670       
predict_under_sample.csv          2021-12-02 18:42:35  Message      complete  0.56639      0.56620       
predict_under_sample.csv          2021-12-02 18:41:43  Message      complete  0.57596      0.57261       
predict_under_sample.csv          2021-12-02 18:41:26  Message      complete  0.57596      0.57216       
predict_under_sample.csv          2021-12-02 18:41:01  Message      complete  0.57668      0.57269       
predict_under.csv                 2021-12-02 1